In [3]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import nlp
import random

In [4]:
def show_history():
       epochs_trained = len(h.history['loss'])
       plt

In [5]:
dataset = nlp.load_dataset('emotion')

Downloading:   0%|          | 0.00/3.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.01k [00:00<?, ?B/s]

Using custom data configuration default


ConnectionError: Couldn't reach https://www.dropbox.com/s/1pzkadrvffbqw6o/train.txt?dl=1